## Load dataset

In [ ]:
import os
import sys
import cv2 as cv
import numpy as np
from matplotlib.pyplot import Rectangle
from videofig import videofig

sys.path.append('..')
from pytracking.pysot_toolkit.datasets import DatasetFactory
from pytracking.pysot_toolkit.environment import env_settings

# set the dataset name here
dataset_name = 'CVPR13'

if dataset_name in ['CVPR13', 'OTB50', 'OTB100']:
    # for OTB datasets, we save results into the same directory
    save_dataset_name = 'OTB100'
else:
    save_dataset_name = dataset_name

dataset_root = os.path.join(env_settings().dataset_path, save_dataset_name)

# load dataset
dataset = DatasetFactory.create_dataset(name=dataset_name, dataset_root=dataset_root, load_img=False)

In [ ]:
dataset.videos.keys()

## Select results to show

In [ ]:
tracker_test_params = 'siamfc.default'
exp_id = 'siamfc.siamfc_alexnet_vid.epoch49'
videoname = 'Bolt'

## Show

In [ ]:
%matplotlib notebook

if 'OTB100' == save_dataset_name:
    filename = '{}.txt'.format(videoname)
elif 'VOT' in save_dataset_name:
    filename = 'baseline/{vname}/{vname}_001.txt'.format(vname=videoname)
else:
    raise NotImplemented
    
video = dataset[videoname]

# load tracking results
boxs = []
with open(os.path.join(env_settings().results_path, save_dataset_name, tracker_test_params, exp_id, filename), 'r') as file_handle:
    for line in file_handle:
        boxs.append([float(v) for v in line.strip().split(',')])

def redraw_fn(f, ax):
    img_path, _  = video[f]
    img = cv.cvtColor(cv.imread(img_path), cv.COLOR_BGR2RGB)
    
    box = boxs[f]
    if len(box) == 4:
        x, y, w, h = box
    else:
        x, y, w, h = 0, 0, 0, 0
          
    if not redraw_fn.initialized:
        redraw_fn.img_handle = ax.imshow(img)
        box_artist = Rectangle((x, y), w, h,
                               fill=False,  # remove background
                               lw=2,
                               edgecolor="red")
        ax.add_patch(box_artist)
        redraw_fn.box_handle = box_artist
        redraw_fn.text_handle = ax.text(0., 1 - 0.05,
                                        'Frame: {}'.format(f + 1),
                                        transform=ax.transAxes,
                                        color='yellow', size=12)
        redraw_fn.initialized = True
    else:
        redraw_fn.img_handle.set_array(img)
        redraw_fn.box_handle.set_xy((x, y))
        redraw_fn.box_handle.set_width(w)
        redraw_fn.box_handle.set_height(h)
        redraw_fn.text_handle.set_text('Frame: {}'.format(f + 1))

redraw_fn.initialized = False

videofig(len(video), redraw_fn)